In [34]:
import numpy as np
import pandas as pd
import random

# Set seed for reproducibility
random.seed(42)
np.random.seed(42)

# --- Settings ---
num_agents = 100
num_weeks = 50
bar_threshold = 65  # global crowd threshold for Average Thinkers
memory_decay_weights_map = {
    2: [0.7, 0.3],
    3: [0.6, 0.3, 0.1],
    4: [0.5, 0.3, 0.15, 0.05]
}

# --- Initialize agents ---
agents = []

for agent_id in range(num_agents):
    strategy = random.choice(['Optimist', 'Pessimist', 'Average'])
    memory_length = random.choice([2, 3, 4])
    agent = {
        'Agent_ID': agent_id,
        'Strategy': strategy,
        'Risk_Tolerance': random.randint(60, 75),
        'Memory_Length': memory_length,
        'Belief': 55  # initial Bayesian prior
    }
    agents.append(agent)

# --- Initialize dataset and history ---
records = []
attendance_history = []

# --- Start Simulation ---
for week in range(1, num_weeks + 1):
    decisions = {}

    # Each agent predicts and decides
    for agent in agents:
        agent_id = agent['Agent_ID']
        strategy = agent['Strategy']
        risk_tolerance = agent['Risk_Tolerance']
        memory_length = agent['Memory_Length']
        belief = agent['Belief']

        # Compute decayed weighted average (for Average thinkers)
        if len(attendance_history) >= memory_length:
            memory = attendance_history[-memory_length:]
            weights = memory_decay_weights_map[memory_length]
            weighted_avg = sum(w * a for w, a in zip(weights, memory)) / sum(weights)
        else:
            weighted_avg = belief

        # Decision based on strategy
        if strategy == 'Optimist':
            # Optimist needs only one past week <= risk tolerance to go
            if len(attendance_history) < memory_length:
                decision = 'Go'
            else:
                memory = attendance_history[-memory_length:]
                decision = 'Go' if any(m <= risk_tolerance for m in memory) else 'Stay'

        elif strategy == 'Pessimist':
            # Pessimist requires all past weeks <= risk tolerance to go
            if len(attendance_history) < memory_length:
                decision = 'Stay'
            else:
                memory = attendance_history[-memory_length:]
                decision = 'Go' if all(m <= risk_tolerance for m in memory) else 'Stay'

        else:  # Average Thinker
            if len(attendance_history) < memory_length:
                decision = 'Go'
            else:
                decision = 'Go' if weighted_avg < bar_threshold else 'Stay'

        decisions[agent_id] = decision

    # Calculate actual attendance
    actual_attendance = sum(1 for d in decisions.values() if d == 'Go')
    attendance_history.append(actual_attendance)

    # Update beliefs
    for agent in agents:
        agent['Belief'] = (agent['Belief'] + actual_attendance) / 2

    # Record data
    for agent_id, decision in decisions.items():
        agent = next(a for a in agents if a['Agent_ID'] == agent_id)
        records.append({
            'Week': week,
            'Agent_ID': agent_id,
            'Strategy': agent['Strategy'],
            'Risk_Tolerance': agent['Risk_Tolerance'],
            'Memory_Length': agent['Memory_Length'],
            'Decision': decision,
            'Actual_Attendance': actual_attendance
        })

# Create DataFrame
df = pd.DataFrame(records)

In [35]:
# Save the DataFrame to CSV
csv_path = "../data/el_farol_simulation.csv"
df.to_csv(csv_path, index=False)

In [36]:
# Reload the CSV to confirm content
df_loaded = pd.read_csv(csv_path)
df_loaded.head()

,Week,Agent_ID,Strategy,Risk_Tolerance,Memory_Length,Decision,Actual_Attendance
0,1,0,Average,60,2,Go,81
1,1,1,Average,67,3,Go,81
2,1,2,Optimist,63,2,Go,81
3,1,3,Average,62,4,Go,81
4,1,4,Average,61,3,Go,81


In [45]:
# Filter the DataFrame for rows where Agent_ID == 1
agent_1_data = df[df["Agent_ID"] == 10]
agent_1_data.head(30)

,Week,Agent_ID,Strategy,Risk_Tolerance,Memory_Length,Decision,Actual_Attendance
10,1,10,Pessimist,64,3,Stay,81
110,2,10,Pessimist,64,3,Stay,81
210,3,10,Pessimist,64,3,Stay,59
310,4,10,Pessimist,64,3,Stay,61
410,5,10,Pessimist,64,3,Stay,60
510,6,10,Pessimist,64,3,Go,82
610,7,10,Pessimist,64,3,Stay,73
710,8,10,Pessimist,64,3,Stay,49
810,9,10,Pessimist,64,3,Stay,48
910,10,10,Pessimist,64,3,Stay,77


In [ ]:
import numpy as np
import pandas as pd
import random

# Set seed for reproducibility
random.seed(42)
np.random.seed(42)

# --- Settings ---
num_agents = 100
num_weeks = 50

# Memory decay weights for different lengths
memory_decay_weights_map = {
    2: [0.7, 0.3],
    3: [0.5, 0.3, 0.2],
    4: [0.4, 0.3, 0.2, 0.1]
}

# --- Initialize agents with random memory lengths and random risk tolerance ---
agents = [{
    'Agent_ID': i,
    'Strategy': random.choice(['Optimist', 'Pessimist', 'Average']),
    'Memory_Length': random.choice([2, 3, 4]),
    'Risk_Tolerance': random.randint(50, 75)  # random tolerance per agent
} for i in range(num_agents)]

attendance_history = []
records = []

# --- Simulation ---
for week in range(1, num_weeks + 1):
    decisions = {}
    
    for agent in agents:
        mem_len = agent['Memory_Length']
        strat = agent['Strategy']
        rt = agent['Risk_Tolerance']
        
        # extract memory slice
        memory = attendance_history[-mem_len:] if len(attendance_history) >= mem_len else []
        weights = memory_decay_weights_map[mem_len]
        
        # default decision if not enough history
        if not memory:
            if strat == 'Optimist':
                decision = 'Go'
            elif strat == 'Pessimist':
                decision = 'Stay'
            else:  # Average
                decision = 'Go' if random.random() < 0.5 else 'Stay'
            A_w = None
        else:
            # compute weighted (decayed) attendance
            A_w = sum(w * a for w, a in zip(weights, memory))
            
            if strat == 'Optimist':
                # go if any one decayed attendance <= own tolerance
                decision = 'Go' if A_w <= rt else 'Stay'
            
            elif strat == 'Pessimist':
                # check trend across memory
                trend_ok = memory[-1] <= (sum(memory[:-1]) / len(memory[:-1]))
                decision = 'Go' if (A_w <= rt and trend_ok) else 'Stay'
            
            else:  # Average Thinker
                decision = 'Go' if A_w <= rt else 'Stay'
        
        decisions[agent['Agent_ID']] = decision
        
        # record this agent-week
        records.append({
            'Week': week,
            'Agent_ID': agent['Agent_ID'],
            'Strategy': strat,
            'Memory_Length': mem_len,
            'Risk_Tolerance': rt,
            'A_weighted': A_w,
            'Decision': decision,
            'Actual_Attendance': None
        })
    
    # compute actual attendance and update history
    actual = sum(1 for d in decisions.values() if d == 'Go')
    attendance_history.append(actual)
    
    # fill actual attendance in records for this week
    for rec in records[-num_agents:]:
        rec['Actual_Attendance'] = actual

# Create DataFrame
df = pd.DataFrame(records)


In [48]:
# Save the DataFrame to CSV
csv_path = "../data/el_farol.csv"
df.to_csv(csv_path, index=False)

In [49]:
# Reload the CSV to confirm content
df_loaded = pd.read_csv(csv_path)
df_loaded.head()

,Week,Agent_ID,Strategy,Memory_Length,A_weighted,Decision,Actual_Attendance
0,1,0,Average,2,NaN,Go,54
1,1,1,Optimist,4,NaN,Go,54
2,1,2,Pessimist,2,NaN,Stay,54
3,1,3,Optimist,2,NaN,Go,54
4,1,4,Average,2,NaN,Stay,54


In [52]:
# Filter the DataFrame for rows where Agent_ID == 1
agent_1_data = df[df["Agent_ID"] == 2]
agent_1_data.head(30)

,Week,Agent_ID,Strategy,Memory_Length,A_weighted,Decision,Actual_Attendance
2,1,2,Pessimist,2,NaN,Stay,54
102,2,2,Pessimist,2,NaN,Stay,54
202,3,2,Pessimist,2,54.0,Stay,58
302,4,2,Pessimist,2,55.2,Stay,38
402,5,2,Pessimist,2,52.0,Stay,70
502,6,2,Pessimist,2,47.6,Stay,58
602,7,2,Pessimist,2,66.4,Stay,37
702,8,2,Pessimist,2,51.7,Stay,58
802,9,2,Pessimist,2,43.3,Stay,60
902,10,2,Pessimist,2,58.6,Stay,55
